Goal:

Make a complex net.

https://github.com/tensorflow/tensorflow/issues/2255

In [1]:
import tensorflow as tf
import functools
from sklearn.datasets import load_digits
digits = load_digits()
from sklearn.preprocessing import OneHotEncoder
labels = OneHotEncoder().fit_transform(digits.target.reshape((digits.target.shape[0],1))).toarray()

In [23]:
d = 64
batch_size = 50
X = tf.placeholder(tf.float32,shape=(None,d))
x = tf.complex(X,tf.zeros((batch_size,d)))
T = tf.placeholder(tf.float32,shape=(None,labels.shape[1]))

In [24]:
def makecomplexvar(name,shape):
    real = tf.get_variable(shape=shape,name=name+'a')
    imag = tf.get_variable(shape=shape,name=name+'b')
    return tf.complex(real,imag)

In [28]:
class Mapping():
    num = 0
    def __init__(self,shape):
        self.name = 'Map'
        self.num = str(Mapping.num)
        Mapping.num+=1
        self.shape = shape
        with tf.variable_scope(self.name+self.num,initializer=tf.truncated_normal_initializer(),reuse=True):
            self.weights = makecomplexvar('W',shape)
            self.biases = makecomplexvar('b',shape[-1])
    
    def __call__(self,x):
        with tf.name_scope(self.name+self.num+'_calc'):
            return tf.matmul(x,self.weights) + self.biases

In [30]:
class LinearNet():
    name = 'LinearNet'
    layers = [Mapping((d,40)),  
              Mapping((40,40)),  
              Mapping((40,40)),  
              Mapping((40,d))]

In [39]:
NN = LinearNet
y = functools.reduce(lambda x,f:f(x),[x] + NN.layers)
#tf.image_summary('Recons',tf.reshape(y,[batch_size,28,28,1]),max_images=10)

def mse(x,y,name='mse'): #is this the right equation to be using?
    with tf.name_scope(name):
        return tf.reduce_mean(tf.reduce_sum(tf.square(tf.complex_abs(x)-tf.complex_abs(y)),reduction_indices=[1]))
    
Loss = mse(x,y)
tf.scalar_summary("Loss",Loss)

train_step = tf.train.AdamOptimizer(0.01).minimize(Loss)
merged = tf.merge_all_summaries()
saver = tf.train.Saver()

In [42]:
def batch_iter(data, batch_size):
    num_batches = data.shape[0] // batch_size
    
    for i in range(num_batches):
        yield data[i*batch_size:(i+1)*batch_size, ...],labels[i*batch_size:(i+1)*batch_size, ...]

In [48]:
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    writer = tf.train.SummaryWriter('Summaries/'+NN.name,sess.graph)
    count = 0
    feed = {}
    for e in range(10):
        for i,batch in enumerate(batch_iter(digits.data,batch_size)):
            ims,lab = batch
            feed[x] = ims.reshape((batch_size,d))/255
            count +=1
            sess.run(train_step,feed_dict=feed)
            if i%100==0:
                summaries = sess.run(merged,feed_dict=feed)
                writer.add_summary(summaries,count)
                print('\r Loss {}'.format(sess.run(Loss,feed)),end='')

InvalidArgumentError: No OpKernel was registered to support Op 'Mean' with these attrs
	 [[Node: Mean = Mean[T=DT_COMPLEX64, keep_dims=false](Sum_1, Const)]]
Caused by op 'Mean', defined at:
  File "/Users/act65/anaconda/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/act65/anaconda/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/act65/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/act65/anaconda/lib/python3.5/site-packages/traitlets/config/application.py", line 596, in launch_instance
    app.start()
  File "/Users/act65/anaconda/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 442, in start
    ioloop.IOLoop.instance().start()
  File "/Users/act65/anaconda/lib/python3.5/site-packages/zmq/eventloop/ioloop.py", line 162, in start
    super(ZMQIOLoop, self).start()
  File "/Users/act65/anaconda/lib/python3.5/